# Goals:

**1. Develop a natural language processor capable of predicting the COLOR of a magic card based on the rules text of that card.**

**2. Develop a natural language processor capable of predicting the TYPE of a magic card based on the rules text of that card.**

In [1]:
# imports and display options

import pandas as pd
import numpy as np
import math
from math import sqrt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, explained_variance_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import unicodedata
import re
import nltk
from nltk.corpus import stopwords

import prepare as p
# import explore as e
# import model as m

pd.set_option('display.max_colwidth', -1)

# Acquire

* Used file from previouse 
* A CSV, containing an up to date breakdown of each card that has been printed so far, was obtained from MTGJSON.com
* Each row represents a card or a version of a card
* The CSV was read into a pandas dataframe
* The original dataframe contained 51,430 rows and 73 columns

# Prepare

The following steps were taken to prepare the data:

1. Restricted dataframe to relevant columns
 
2. Restricted dataframe to rows containing cards that exist in physical form

3. Restricted dataframe to rows containing a value for 'text'

4. Restricted dataframe to rows with a single 'color identity' (see data dictionary Color)

5. Dropped all columns other than text and color

6. Renamed columns

7. Changed some of the symble represintations to words to make them machine readable

8. Applied basic cleaning to values in text by, lowercasing letters, converting to ASCII characters, removing non-letter characters, lemmatizeing the words, and removing stopwords

9. Dropped duplicate rows

10. Changed column order

11. Wrote prepared data to ‘mtgprep.csv’ for ease of access. Data consisted of 15380 rows and two columns.

12. Created a test and train group at a 20/80 split

## Data Dictionary:

* Magic: The Gathering
    * a collectable card game developed by Wizards of the Coast Inc. In a typical game, each player combines a selection of cards from their collection into his or her own deck, which they use to compete against other players. Thematically, players assume the roles of powerful mages fighting for supremacy. Cards in each players deck represent spells and other resources each player’s disposal. A game typically ends when all but one of the participating players is reduced to zero “life” or is otherwise eliminated from the game.

### Columns

* Color
    * Spells players can cast are divided into five different colors Each color is thematically distinct
    * White represents order and morality
    * Blue represents cunning and technology
    * Black represents pragmatism and amorality
    * Red represents impulse and chaos
    * Green represents nature and instinct
    * Spells with more than one color were excluded for the study to give a clearer picture of sentiment values for each color
    * The color or colors each spell had was determined by the color of mana symbols that appear on each card, also called its color identity
    <br />   
    
    
* Text
    * The value in text represents the rules text appearing on one Magic the Gathering card

In [2]:
df = p.get_preped_data()

In [3]:
df.shape

(15380, 2)

In [5]:
df

,color,text
0,Green,would draw card may instead choose land nonland reveal card top library reveal card chosen kind put card hand put card revealed way bottom library order
1,Blue,academy researcher enters battlefield may put aura card hand onto battlefield attached academy researcher
3,Black,target creature get minusandminus end turn draw card
4,Green,target creature get plusandplus end turn draw card
5,Black,look target player hand choose two card put top player library order
...,...,...
51412,Blue,flying whenever umara raptor another ally enters battlefield control may put plusandplus counter umara raptor
51413,Red,kicker redmana may pay additional redmana cast spell damage prevented turn spell kicked deal damage target player planeswalker
51427,White,beginning upkeep may choose card type player sacrifice permanent type
51428,Red,landfall whenever land enters battlefield control may put quest counter zektar shrine expedition remove three quest counter zektar shrine expedition sacrifice create red elemental creature token trample haste exile beginning next end step
